In [1]:
import pickle
import os
from pathlib import Path
from bs4 import BeautifulSoup
from parse_utils import get_title, get_dates, get_doc_type, get_rubrics
import re
from tqdm.auto import tqdm

In [2]:
# # current directory
# current_directory = Path(os.getcwd())

# # parent directory
# parent_directory = current_directory.parent
# parent_directory

In [3]:
with open("./data/docs.pkl", "rb") as f:
    docs = pickle.load(f)

href2doc = {doc["href"]: doc for doc in docs}
doc = href2doc["https://company.rzd.ru/ru/9353/page/105104?id=1604"]

soup = BeautifulSoup(doc["html"], "html.parser")

all_links = soup.find_all("a")
all_hrefs = [link.attrs['href'] for link in all_links]

doc["title"] = get_title(soup)
doc["dates"] = get_dates(soup)
doc["doc_type"], doc["doc_date"] = get_doc_type(soup)
doc["rubrics"] = get_rubrics(soup)

soup = soup.find("div", {"class": "attached"})

# Для удобства чтения 
for el in soup.find_all("div", {'class':'print-hide'}):
    el.decompose()

# Для удобства чтения 
for el in soup.find_all("a", {'class':'goto-rubrics'}):
    # print(el.text.strip())
    el.decompose()

In [4]:
# parts = soup.find_all("div", {"class": "static-content"})
elements = soup.select(".attached > .static-content")
elements = [elem for elem in elements if len(elem.text.replace(" ", "")) > 3] # get('id') and elem['id'].strip()

print("Elements:", len(elements))

if len(elements) == 0:
    # Ищем ссылку где есть ?action=download
    # links = soup.find_all("a", {"href": True})
    # hrefs = [link['href'] for link in all_links]
    hrefs = [href for href in all_hrefs if "action=download" in href]
    hrefs = ["https://company.rzd.ru" + href for href in hrefs]

    print("Hrefs:", hrefs)

else:
    attachments = []
    parts = []
    for part in elements:
        part_title = part.find("h3")
        if part_title is None:
            part_title = ""
        else:
            part_title = part_title.text.strip()

        contents = part.find_all("div", {"class": "onePartTextOut_text"})
        # print(len(contents))

        if part_title.lower() == 'приложения':
            attachments.append(part)
        else:
            parts.append(part)

    print("Len parts:", len(parts))
    print("Len attachments:", len(attachments))


Elements: 13
Len parts: 13
Len attachments: 0


In [5]:
chunks = []

for part in parts:
    h3 = part.find("h3")
    if h3 is None:
        continue

    part_title = h3.text.strip()

    content = part.find("div", {"class": "onePartTextOut_text"})

    paragraphs = content.find_all("p")
    paragraphs = [p.text.strip() for p in paragraphs]

    for paragraph in paragraphs:
        chunks.append({
            "doc_title": doc["title"],
            "doc_href": doc["href"],
            "doc_type": doc["doc_type"],
            "doc_date": doc["doc_date"],
            "doc_rubrics": doc["rubrics"],
            "part_title": part_title,
            "text": paragraph
        })

In [6]:
from llm_utils import generate_summary_questions_keywords_for_chunk

for chunk in chunks:
    text = chunk["text"]
    context = f"""Название документа: {chunk["doc_title"]}
Название раздела: {chunk["part_title"]}
Тип документа: {chunk["doc_type"]}
Рубрики документа: {", ".join(chunk["doc_rubrics"])}
"""
    data = generate_summary_questions_keywords_for_chunk(text, context)
    break

In [58]:
texts = [c["text"] for c in chunks]

In [ ]:
import numpy as np
from text_embedding import get_embedding

texts_vecs = []
for text in tqdm(texts):
    texts_vecs.append(get_embedding(text))

array([ 3.1250000e-02, -4.8583984e-02, -3.1433105e-03, ...,
       -8.8500977e-03,  3.7550926e-06,  2.6245117e-02], dtype=float32)

In [45]:
parts[1]

<div class="docTextPartDiv static-content" id="7274">
<h3 class="parttext">Раздел 1. Основные понятия</h3>
<div class="onePartTextOut_text"><p>Коллективный договор ОАО «РЖД» на 2023-2025 годы – правовой акт, регулирующий социально-трудовые отношения в открытом акционерном обществе «Российские железные дороги» между сторонами социального партнерства – Работниками и Работодателем в лице их представителей.</p>
<p>Настоящий Договор является единым для ОАО «РЖД», включая филиалы, структурные подразделения и представительства, за исключением Петропавловского отделения Южно-Уральской железной дороги – филиала ОАО «РЖД», расположенного на территории Республики Казахстан, в котором, на основе настоящего Договора и с учетом особенностей законодательства Республики Казахстан, заключается отдельный коллективный договор на 2023-2025 годы.</p>
<p>Предоставление Работникам одних и тех же гарантий, льгот и компенсаций в соответствии с Договором и коллективным договором указанного филиала не допускаетс

In [38]:
html = soup.prettify()
print(html)

<div class="attached">
 <div class="static-content">
  <p>
   (утвержден Распоряжением ОАО «РЖД» №3446 от 26 декабря 2022 г.)
  </p>
 </div>
 <div class="docTextPartDiv static-content" id="7274">
  <h3 class="parttext">
   Раздел 1. Основные понятия
  </h3>
  <div class="onePartTextOut_text">
   <p>
    Коллективный договор ОАО «РЖД» на 2023-2025 годы – правовой акт, регулирующий социально-трудовые отношения в открытом акционерном обществе «Российские железные дороги» между сторонами социального партнерства – Работниками и Работодателем в лице их представителей.
   </p>
   <p>
    Настоящий Договор является единым для ОАО «РЖД», включая филиалы, структурные подразделения и представительства, за исключением Петропавловского отделения Южно-Уральской железной дороги – филиала ОАО «РЖД», расположенного на территории Республики Казахстан, в котором, на основе настоящего Договора и с учетом особенностей законодательства Республики Казахстан, заключается отдельный коллективный договор на 2023

In [28]:
with open("index.html", "w") as f:
    f.write(html)

In [29]:
with open("index.html", "r") as f:
    html = f.read()

html

'<div class="docs box-wrap doc-tab">\n <div class="date-top">\n  <span class="text-more-grey">\n   Дата официальной публикации\n  </span>\n  27.12.2022\n </div>\n <div class="date-top">\n  <span class="text-more-grey">\n   Дата вступления в силу\n  </span>\n  01.01.2023\n </div>\n <h2 class="card-header">\n  Коллективный договор открытого акционерного общества «Российские железные дороги»\n                     на 2023-2025 годы\n </h2>\n <div class="doc-type">\n  Договор\n  <span>\n   ОАО «РЖД»\n  </span>\n  <span>\n   от 26.12.2022\n  </span>\n </div>\n <div class="rubrics">\n  <a class="text-more-grey text-small" href="/ru/9353/page/105103?rubrics=109">\n   Деятельность ОАО «РЖД»\n  </a>\n  <a class="text-more-grey text-small" href="/ru/9353/page/105103?rubrics=120">\n   Сотрудникам\n  </a>\n  <a class="text-more-grey text-small" href="/ru/9353/page/105103?rubrics=121">\n   Трудовая деятельность\n  </a>\n </div>\n <hr/>\n <div class="attached">\n  <div class="static-content">\n   <p>

In [30]:
from langchain_community.document_loaders import BSHTMLLoader

loader = BSHTMLLoader("index.html")
data = loader.load()

print(data)

[Document(page_content='\n\n\n   Дата официальной публикации\n  \n  27.12.2022\n \n\n\n   Дата вступления в силу\n  \n  01.01.2023\n \n\n  Коллективный договор открытого акционерного общества «Российские железные дороги»\n                     на 2023-2025 годы\n \n\n  Договор\n  \n   ОАО «РЖД»\n  \n\n   от 26.12.2022\n  \n\n\n\n   Деятельность ОАО «РЖД»\n  \n\n   Сотрудникам\n  \n\n   Трудовая деятельность\n  \n\n\n\n\n\n    (утвержден Распоряжением ОАО «РЖД» №3446 от 26 декабря 2022 г.)\n   \n\n\n\n    Раздел 1. Основные понятия\n   \n\n\n     Коллективный договор ОАО «РЖД» на 2023-2025 годы – правовой акт, регулирующий социально-трудовые отношения в открытом акционерном обществе «Российские железные дороги» между сторонами социального партнерства – Работниками и Работодателем в лице их представителей.\n    \n\n     Настоящий Договор является единым для ОАО «РЖД», включая филиалы, структурные подразделения и представительства, за исключением Петропавловского отделения Южно-Уральской ж